In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [ ]:
# 먼저 문서를 작은 chunk로 나누는 "map" 단계와, 각 chunk의 요약을 결합하는 "reduce" 단계로 구성됩니다.

In [2]:
# 1. Map 단계에서는 각 chunk를 병렬로 요약하고
# 2. reduce 단계에서는 이 요약들을 하나의 최종 요약으로 통합합니다.

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

In [6]:
docs = docs[3:8]  # 여기서 문서의 일부만 요약
print(f"총 페이지수: {len(docs)}")

총 페이지수: 5


In [7]:
# map 단계에서는 각 Chunk 에 대한 요약을 생성합니다.

In [11]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

In [12]:
llm = ChatOpenAI(
    api_key=key, 
    model_name='gpt-4o-mini',
    temperature=0
)

In [13]:
map_prompt = hub.pull("teddynote/map-prompt")   # map prompt 다운로드

In [14]:
map_prompt.pretty_print()       # 프롬프트 출력

================================ System Message ================================

You are a professional main thesis extractor.

================================ Human Message =================================

Your task is to extract main thesis from given documents. Answer should be in same language as given document. 

#Format: 
- thesis 1
- thesis 2
- thesis 3
- ...

Here is a given document: 
{doc}

Write 1~5 sentences.
#Answer:


In [18]:
# map_chain 을 생성

In [19]:
map_chain = map_prompt | llm | StrOutputParser()

In [20]:
# batch() 를 호출하여 각 문서에 대한 요약본을 생성합니다.

In [21]:
doc_summaries = map_chain.batch(docs)       # 문서에 대한 주요내용 추출

In [24]:
len(doc_summaries)  # 요약된 문서의 수 출력

5

In [25]:
print(doc_summaries[0])     # 일부 문서의 요약 출력

- 미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행정명령을 발표하였다.
- 이 행정명령은 AI의 안전과 보안 기준 마련, 개인정보보호, 형평성과 시민권 향상, 소비자 보호, 노동자 지원, 혁신과 경쟁 촉진, 국제협력을 포함한다.
- AI 시스템의 안전성과 신뢰성을 확인하기 위한 표준 및 모범사례를 확립하고, AI의 무책임한 사용으로 인한 차별과 편견을 방지하기 위한 조치를 확대할 예정이다.
- 또한, 의료 분야에서 책임 있는 AI 사용을 촉진하고, 중소기업과 개발자에게 기술과 인프라를 지원하여 AI 연구를 촉진할 계획이다.


In [26]:
# Reduce 단계에서는 map 단계에서 진행한 핵심 내용들을 하나의 최종 요약으로 통합합니다.

In [ ]:
reduce_prompt = hub.pull("teddynote/reduce-prompt")     # reduce prompt 다운로드

In [29]:
reduce_prompt.pretty_print()                            # 프롬프트 출력

================================ System Message ================================

You are a professional summarizer. You are given a list of summaries of documents and you are asked to create a single summary of the documents.

================================ Human Message =================================

#Instructions: 
1. Extract main points from a list of summaries of documents
2. Make final summaries in bullet points format.
3. Answer should be written in {language}.

#Format: 
- summary 1
- summary 2
- summary 3
- ...

Here is a list of summaries of documents: 
{doc_summaries}

#SUMMARY:


In [30]:
# Reduce Chain 을 생성합니다.

In [32]:
# reduce chain 생성
reduce_chain = reduce_prompt | llm | StrOutputParser()

In [34]:
# Reduce Chain 을 사용하여 스트리밍 출력 

In [ ]:
from langchain_teddynote.messages import stream_response

In [35]:
answer = reduce_chain.stream(
    {"doc_summaries": "\n".join(doc_summaries), "language": "Korean"}
)

In [36]:
stream_response(answer)

- 미국 바이든 대통령이 AI 개발과 사용의 안전성을 보장하기 위한 행정명령을 발표하였다.
- 행정명령은 AI의 안전 기준, 개인정보 보호, 형평성, 소비자 보호, 노동자 지원, 혁신 촉진, 국제 협력을 포함한다.
- AI 시스템의 안전성과 신뢰성을 위한 표준을 확립하고, 차별과 편견 방지를 위한 조치를 확대할 예정이다.
- G7은 AI 위험 식별과 완화를 위한 국제 행동강령을 마련하고, AI 시스템의 투명성을 강화할 계획이다.
- 영국 AI 안전성 정상회의에서 AI 안전 보장을 위한 블레츨리 선언이 발표되었으며, AI 안전 연구소의 출범이 발표되었다.
- 미국 법원은 AI 생성 이미지에 대한 저작권 침해 소송을 기각하였으나, 일부 소송은 계속 진행될 예정이다.
- 미국 연방거래위원회(FTC)는 생성 AI의 소비자 피해와 빅테크의 시장 지배력 강화에 대한 우려를 표명하였다.